In [ ]:
import os, sys
project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

attention_dir = os.path.join(project_dir, 'modules/AttentionMap')
if attention_dir not in sys.path:
    sys.path.append(attention_dir)

sparse_dir = os.path.join(project_dir, 'modules/Sparse')
if sparse_dir not in sys.path:
    sys.path.append(sparse_dir) 

import numpy as np
import config, torch
from torch import nn
import os
from derma.dataset import Derma, get_samples_weight

In [ ]:
from torchvision.transforms import Compose, ToTensor, Normalize, RandomHorizontalFlip, RandomVerticalFlip, RandomRotation
transform = Compose([
        RandomHorizontalFlip(), 
        RandomVerticalFlip(),
        RandomRotation(90),
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

dataset = Derma(config.DATASET_DIR, transform=transform)
dataset2 = Derma(config.DATASET_DIR, transform=transform)

from torch.utils.data import DataLoader, random_split
val_size = int(0.1*len(dataset))
train_size = len(dataset) - val_size

train_set, val_set = random_split(dataset, [train_size, val_size])
val_set.dataset = dataset2

train_sampler, _ = get_samples_weight(train_set)
train_loader = DataLoader(train_set, batch_size=64, shuffle=False, sampler=train_sampler)

val_sampler, _ = get_samples_weight(val_set)
val_loader = DataLoader(val_set, batch_size=512, shuffle=False, sampler=val_sampler)

In [ ]:
from derma.architecture import InvertedResidual
from torchvision.models import MobileNetV2

inverted_residual_setting = [
        # t, c, n, s
        [1, 16, 1, 1],
        [6, 24, 2, 2],
        [6, 32, 3, 2],
        [6, 64, 4, 2],
        [6, 96, 3, 1],
        [6, 160, 3, 2],
        [6, 320, 1, 1],
    ]

# inverted_residual_setting = [
#         # t, c, n, s
#         [1, 16, 1, 1],
#         [2, 24, 1, 2],
#         [2, 32, 1, 2],
#         [2, 64, 1, 2],
#         [2, 96, 1, 1],
#         [2, 160, 1, 2],
#         [2, 320, 1, 1],
#     ]

labels = [0, 1]
model = MobileNetV2(num_classes=len(labels), inverted_residual_setting=inverted_residual_setting, block=InvertedResidual)
# model = MobileNetV2(num_classes=len(labels), inverted_residual_setting=inverted_residual_setting)

In [ ]:
from derma.utils import train
from torch.utils.tensorboard import SummaryWriter

optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-6)
tb_writer = SummaryWriter(log_dir='test/CoordAtt')
criterion = torch.nn.CrossEntropyLoss()

train(model, [train_loader, val_loader], optimizer, criterion, 10, tb_writer)

In [ ]:
inverted_residual_setting = [
        # t, c, n, s
        [1, 16, 1, 1],
        [6, 24, 2, 2],
        [6, 32, 3, 2],
        [6, 64, 4, 2],
        [6, 96, 3, 1],
        [6, 160, 3, 2],
        [6, 320, 1, 1],
    ]

encoder = MobileNetV2(num_classes=1000, inverted_residual_setting=inverted_residual_setting).features

In [ ]:
inverted_residual_setting = [
        # t, c, n, s
        [1, 16, 1, 1],
        [6, 24, 2, 2],
        [6, 32, 3, 2],
        [6, 64, 4, 2],
        [6, 96, 3, 1],
        [6, 160, 3, 2],
        [6, 320, 1, 1],
    ]

'si s==2 se reduce a la mitad la imagen'
a = torch.rand((1, 3, 128, 128))
print(encoder[:2](a).shape)
print(encoder[:4](a).shape)
print(encoder[:7](a).shape)
print(encoder[:11](a).shape)
print(encoder[:14](a).shape)
print(encoder[:17](a).shape)
print(encoder[:18](a).shape)


In [ ]:
from torch import nn
def create_conv(in_channels, out_channels, upsampling=True):
    if upsampling:
        return nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )
    else:
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )


from collections import deque
from itertools import islice

def sliding_window_iter(iterable, size):
    '''
        Iterate through iterable using a sliding window of several elements.
        Important: It is a generator!.
        
        Creates an iterable where each element is a tuple of `size`
        consecutive elements from `iterable`, advancing by 1 element each
        time. For example:
        >>> list(sliding_window_iter([1, 2, 3, 4], 2))
        [(1, 2), (2, 3), (3, 4)]
        
        source: https://codereview.stackexchange.com/questions/239352/sliding-window-iteration-in-python
    '''
    iterable = iter(iterable)
    window = deque(islice(iterable, size), maxlen=size)
    for item in iterable:
        yield tuple(window)
        window.append(item)
    if window:  
        # needed because if iterable was already empty before the `for`,
        # then the window would be yielded twice.
        yield tuple(window)

layers = []
for setting in sliding_window_iter(inverted_residual_setting[::-1], 2):
    in_setting, out_setting = setting
    _, in_c, _, _ = in_setting
    _, out_c, _, out_s = out_setting
    
    upsample = True if out_s == 2 else False
    layers.append(create_conv(in_c, out_c, upsample))

out_layer = nn.Sequential(
    nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
    nn.Conv2d(16, 3, kernel_size=3, stride=1, padding=1),
)
layers.append(out_layer)
result = nn.Sequential(*layers)
    

In [ ]:
from torch import nn
test = torch.rand(1, 320, 4, 4)
result(test).shape

In [ ]:
inverted_residual_setting[:-1][::-1]

In [ ]:
from torchvision.models import MobileNetV2
from derma.architecture import MobileNetDecoder

inverted_residual_setting = [
        # t, c, n, s
        [1, 16, 1, 1],
        [6, 24, 2, 2],
        [6, 32, 3, 2],
        [6, 64, 4, 2],
        [6, 96, 3, 1],
        [6, 160, 3, 2],
        [6, 320, 1, 1],
    ]

encoder = MobileNetV2(inverted_residual_setting=inverted_residual_setting).features
decoder = MobileNetDecoder(inverted_residual_setting)
model = nn.Sequential(encoder, decoder)

In [ ]:
from derma.utils import train
from torch.utils.tensorboard import SummaryWriter

optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-6)
tb_writer = SummaryWriter(log_dir='reconstruction/')
criterion = nn.MSELoss()

train(model, [train_loader, val_loader], optimizer, criterion, 10, tb_writer, reconstruction=True)